# Walk-Forward DLinear with Regime Detection

This notebook demonstrates how to use the walk-forward validation framework with regime detection to train and evaluate DLinear models on crypto market data.

## Key Components:
- **Walk-Forward Validation**: Creates chronological folds with train/val/test splits
- **Regime Detection**: Clusters market conditions (e.g., high/low volatility) using K-Means
- **DLinear Model**: Decomposition-based time series forecasting
- **Per-Fold Training**: Trains separate models on each fold to simulate realistic deployment

## Pipeline Overview:
```
Raw 5m OHLCV → Aggregate to 1h → Engineer Features → Detect Regimes → 
Generate Walk-Forward Folds → Train DLinear on Each Fold → Evaluate Performance
```

In [ ]:
# Commented out IPython magic to ensure Python compatibility.
# 2. Clone the repository
!git clone https://github.com/tensorlink-dev/open-synth-miner
# %cd open-synth-miner
!uv pip install torchsde
# 3. Install dependencies using uv
# --system: Installs into the Colab runtime (no venv needed)
# -e .: Installs the package in editable mode
!uv pip install --system -e .

# 4. (Optional) Verify installation
!python main.py --help

In [ ]:
# Commented out IPython magic to ensure Python compatibility.
# %cd open-synth-miner
!git pull
!uv pip install --system -e .

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

# Set style
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (14, 6)

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 1. Import Walk-Forward and Regime Components

In [ ]:
from src.data.regime_loader import (
    load_raw_data,
    aggregate_5m_to_1h,
    engineer_features,
    RegimeTagger,
    KMeansStrategy,
    GaussianMixtureStrategy,
    generate_walk_forward_folds,
    RegimeBalancedSampler,
    RegimeDriftMonitor,
    PipelineConfig,
    run_pipeline
)

from src.models.components.advanced_blocks import DLinearBlock
from src.models.components.backbones import HybridBackbone
from src.models.heads import HorizonHead

## 2. Load and Prepare Data

We'll load raw 5-minute OHLCV data and process it through the regime detection pipeline.

In [ ]:
# Load raw 5-minute data
# Option 1: From HuggingFace (recommended)
df_raw = load_raw_data(
    repo_id="tensorlink-dev/open-synth-training-data",
    asset="BTC_USD",
    date_range=("2023-01-01", "2024-12-31")
)

# Option 2: From local file
# df_raw = pd.read_parquet('data/BTC_USD/5m.parquet')
# df_raw.index = pd.to_datetime(df_raw.index)

print(f"Loaded {len(df_raw)} bars from {df_raw.index[0]} to {df_raw.index[-1]}")
df_raw.head()

## 3. Configure Walk-Forward Pipeline

Set up the pipeline parameters for:
- Aggregation (5m → 1h)
- Feature engineering (volatility, momentum, microstructure)
- Regime detection (K-Means clustering)
- Walk-forward fold generation

In [ ]:
# Configure pipeline
config = PipelineConfig(
    # Fold sizes (in hours for 1h bars)
    train_size=720,      # 30 days
    val_size=168,        # 7 days
    test_size=168,       # 7 days
    step_size=168,       # Slide forward by 7 days
    gap_size=24,         # 1-day gap between train/val and val/test
    
    # Model input/output dimensions
    seq_len=64,          # 64 hours lookback (2.67 days)
    pred_len=12,         # 12 hours forecast (0.5 days)
    
    # Regime detection
    n_regimes=3,         # Low/Medium/High volatility
    regime_features=['realized_vol', 'parkinson_vol'],
    clustering_strategy='kmeans',  # or 'gmm'
    
    # DataLoader settings
    batch_size=32,
    num_workers=0,
    use_regime_balancing=True,  # Oversample minority regimes
)

print("Pipeline Configuration:")
print(f"  Train: {config.train_size}h ({config.train_size/24:.1f} days)")
print(f"  Val:   {config.val_size}h ({config.val_size/24:.1f} days)")
print(f"  Test:  {config.test_size}h ({config.test_size/24:.1f} days)")
print(f"  Gap:   {config.gap_size}h ({config.gap_size/24:.1f} days)")
print(f"  Step:  {config.step_size}h ({config.step_size/24:.1f} days)")
print(f"  Regimes: {config.n_regimes}")

## 4. Run Pipeline to Generate Folds

This will:
1. Aggregate 5m → 1h bars
2. Engineer 13 features (volatility, momentum, microstructure)
3. Detect regimes using K-Means clustering
4. Generate walk-forward folds with DataLoaders

In [ ]:
# Run the full pipeline
fold_loaders = run_pipeline(df_raw, config)

print(f"\nGenerated {len(fold_loaders)} walk-forward folds\n")

# Inspect first fold
for i, fold_loader in enumerate(fold_loaders[:3]):  # Show first 3 folds
    train_dl, val_dl, test_dl = fold_loader.train_dl, fold_loader.val_dl, fold_loader.test_dl
    
    print(f"Fold {i+1}:")
    print(f"  Train: {len(train_dl.dataset)} samples")
    print(f"  Val:   {len(val_dl.dataset)} samples")
    print(f"  Test:  {len(test_dl.dataset)} samples")
    
    # Show regime distribution
    if hasattr(fold_loader, 'regime_distribution'):
        dist = fold_loader.regime_distribution()
        print(f"  Regime distribution: {dist}")
    print()

## 5. Visualize Regime Detection

Plot the detected regimes over time to understand market state transitions.

In [ ]:
# Extract regime labels from first fold for visualization
first_fold = fold_loaders[0]
df_first = first_fold.df_train  # DataFrame with timestamp index

if 'regime' in df_first.columns:
    fig, axes = plt.subplots(3, 1, figsize=(16, 10), sharex=True)
    
    # Plot 1: Price with regime background
    ax1 = axes[0]
    ax1.plot(df_first.index, df_first['close'], color='black', linewidth=1)
    
    # Color background by regime
    regime_colors = {0: 'green', 1: 'yellow', 2: 'red'}
    for regime_id in df_first['regime'].unique():
        regime_mask = df_first['regime'] == regime_id
        ax1.fill_between(
            df_first.index, 
            df_first['close'].min(), 
            df_first['close'].max(),
            where=regime_mask,
            alpha=0.2,
            color=regime_colors.get(regime_id, 'gray'),
            label=f'Regime {regime_id}'
        )
    
    ax1.set_ylabel('Close Price')
    ax1.set_title('Price with Regime Background (Fold 1 Training Data)')
    ax1.legend(loc='upper left')
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Realized Volatility
    ax2 = axes[1]
    if 'realized_vol' in df_first.columns:
        ax2.plot(df_first.index, df_first['realized_vol'], color='purple', linewidth=1)
        ax2.set_ylabel('Realized Vol')
        ax2.set_title('Realized Volatility Over Time')
        ax2.grid(True, alpha=0.3)
    
    # Plot 3: Regime timeline
    ax3 = axes[2]
    ax3.scatter(df_first.index, df_first['regime'], c=df_first['regime'], 
                cmap='RdYlGn_r', s=10, alpha=0.6)
    ax3.set_ylabel('Regime ID')
    ax3.set_xlabel('Time')
    ax3.set_title('Regime Timeline')
    ax3.set_yticks([0, 1, 2])
    ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
else:
    print("No regime column found in data")

## 6. Build DLinear Model

Create a DLinear model with:
- Trend-seasonal decomposition
- Separate linear projections for each component
- Multi-horizon forecasting head

In [ ]:
def create_dlinear_model(input_size, d_model=64, kernel_size=25, pred_len=12, num_paths=1):
    """Create a DLinear model with specified architecture.
    
    Args:
        input_size: Number of input features
        d_model: Hidden dimension size
        kernel_size: Moving average kernel for decomposition
        pred_len: Forecast horizon length
        num_paths: Number of simulation paths (1 for point forecast)
    
    Returns:
        nn.Module: Complete DLinear model
    """
    # Build backbone with stacked DLinear blocks
    blocks = [
        DLinearBlock(d_model=d_model, kernel_size=kernel_size),
        DLinearBlock(d_model=d_model, kernel_size=kernel_size),
    ]
    
    backbone = HybridBackbone(
        input_size=input_size,
        d_model=d_model,
        blocks=blocks,
    )
    
    # Add forecasting head
    head = HorizonHead(
        d_model=d_model,
        pred_len=pred_len,
        num_paths=num_paths,
    )
    
    class DLinearModel(nn.Module):
        def __init__(self, backbone, head):
            super().__init__()
            self.backbone = backbone
            self.head = head
        
        def forward(self, x):
            features = self.backbone(x)
            return self.head(features)
    
    return DLinearModel(backbone, head)

# Get input size from first batch
sample_batch = next(iter(fold_loaders[0].train_dl))
input_size = sample_batch['features'].shape[-1]
print(f"Input features: {input_size}")
print(f"Prediction horizon: {config.pred_len}")

# Create model
model = create_dlinear_model(
    input_size=input_size,
    d_model=64,
    kernel_size=25,
    pred_len=config.pred_len,
    num_paths=1
).to(device)

print(f"\nModel architecture:")
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")

## 7. Training Function

Define training loop with:
- MSE loss (can be replaced with CRPS for probabilistic forecasts)
- Adam optimizer
- Validation monitoring

In [ ]:
def train_model(model, train_dl, val_dl, epochs=10, lr=1e-3, patience=3):
    """Train model on a single fold.
    
    Args:
        model: PyTorch model
        train_dl: Training DataLoader
        val_dl: Validation DataLoader
        epochs: Number of training epochs
        lr: Learning rate
        patience: Early stopping patience
    
    Returns:
        dict: Training history
    """
    optimizer = Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    
    history = {'train_loss': [], 'val_loss': []}
    best_val_loss = float('inf')
    patience_counter = 0
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_losses = []
        
        for batch in tqdm(train_dl, desc=f"Epoch {epoch+1}/{epochs} [Train]", leave=False):
            features = batch['features'].to(device)
            targets = batch['targets'].to(device)
            
            optimizer.zero_grad()
            predictions = model(features)
            
            # Handle potential shape mismatch
            if predictions.dim() == 3 and targets.dim() == 2:
                predictions = predictions.squeeze(-1)
            
            loss = criterion(predictions, targets)
            loss.backward()
            optimizer.step()
            
            train_losses.append(loss.item())
        
        # Validation
        model.eval()
        val_losses = []
        
        with torch.no_grad():
            for batch in tqdm(val_dl, desc=f"Epoch {epoch+1}/{epochs} [Val]", leave=False):
                features = batch['features'].to(device)
                targets = batch['targets'].to(device)
                
                predictions = model(features)
                
                if predictions.dim() == 3 and targets.dim() == 2:
                    predictions = predictions.squeeze(-1)
                
                loss = criterion(predictions, targets)
                val_losses.append(loss.item())
        
        # Record metrics
        train_loss = np.mean(train_losses)
        val_loss = np.mean(val_losses)
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        
        print(f"Epoch {epoch+1}: Train Loss = {train_loss:.6f}, Val Loss = {val_loss:.6f}")
        
        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break
    
    return history

## 8. Walk-Forward Training Loop

Train a separate model on each fold and evaluate on the test set.
This simulates a realistic deployment scenario where models are retrained periodically.

In [ ]:
# Store results for each fold
fold_results = []

# Train on each fold (limit to first 5 folds for demo)
num_folds = min(5, len(fold_loaders))

for fold_idx in range(num_folds):
    print(f"\n{'='*80}")
    print(f"FOLD {fold_idx + 1} / {num_folds}")
    print(f"{'='*80}\n")
    
    fold_loader = fold_loaders[fold_idx]
    
    # Create fresh model for this fold
    model = create_dlinear_model(
        input_size=input_size,
        d_model=64,
        kernel_size=25,
        pred_len=config.pred_len,
        num_paths=1
    ).to(device)
    
    # Train model
    history = train_model(
        model,
        fold_loader.train_dl,
        fold_loader.val_dl,
        epochs=15,
        lr=1e-3,
        patience=5
    )
    
    # Evaluate on test set
    model.eval()
    test_losses = []
    all_predictions = []
    all_targets = []
    
    with torch.no_grad():
        for batch in fold_loader.test_dl:
            features = batch['features'].to(device)
            targets = batch['targets'].to(device)
            
            predictions = model(features)
            
            if predictions.dim() == 3 and targets.dim() == 2:
                predictions = predictions.squeeze(-1)
            
            loss = nn.MSELoss()(predictions, targets)
            test_losses.append(loss.item())
            
            all_predictions.append(predictions.cpu().numpy())
            all_targets.append(targets.cpu().numpy())
    
    test_loss = np.mean(test_losses)
    all_predictions = np.concatenate(all_predictions, axis=0)
    all_targets = np.concatenate(all_targets, axis=0)
    
    # Calculate additional metrics
    mae = np.mean(np.abs(all_predictions - all_targets))
    rmse = np.sqrt(np.mean((all_predictions - all_targets) ** 2))
    
    print(f"\nTest Metrics:")
    print(f"  MSE:  {test_loss:.6f}")
    print(f"  MAE:  {mae:.6f}")
    print(f"  RMSE: {rmse:.6f}")
    
    # Store results
    fold_results.append({
        'fold': fold_idx + 1,
        'history': history,
        'test_loss': test_loss,
        'mae': mae,
        'rmse': rmse,
        'predictions': all_predictions,
        'targets': all_targets
    })

print(f"\n{'='*80}")
print("WALK-FORWARD TRAINING COMPLETE")
print(f"{'='*80}\n")

## 9. Aggregate Results Across Folds

In [ ]:
# Create results summary
results_df = pd.DataFrame([{
    'Fold': r['fold'],
    'Test MSE': r['test_loss'],
    'Test MAE': r['mae'],
    'Test RMSE': r['rmse']
} for r in fold_results])

print("\nPer-Fold Results:")
print(results_df.to_string(index=False))

print("\nAggregate Statistics:")
print(f"  Mean Test MSE:  {results_df['Test MSE'].mean():.6f} ± {results_df['Test MSE'].std():.6f}")
print(f"  Mean Test MAE:  {results_df['Test MAE'].mean():.6f} ± {results_df['Test MAE'].std():.6f}")
print(f"  Mean Test RMSE: {results_df['Test RMSE'].mean():.6f} ± {results_df['Test RMSE'].std():.6f}")

## 10. Visualize Results

In [ ]:
# Plot 1: Training curves for all folds
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

ax1 = axes[0]
for i, result in enumerate(fold_results):
    history = result['history']
    ax1.plot(history['train_loss'], label=f"Fold {i+1} Train", alpha=0.7)
    ax1.plot(history['val_loss'], label=f"Fold {i+1} Val", linestyle='--', alpha=0.7)

ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.set_title('Training Curves Across Folds')
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax1.grid(True, alpha=0.3)

# Plot 2: Test metrics comparison
ax2 = axes[1]
x = results_df['Fold']
ax2.bar(x - 0.2, results_df['Test MSE'], width=0.2, label='MSE', alpha=0.8)
ax2.bar(x, results_df['Test MAE'], width=0.2, label='MAE', alpha=0.8)
ax2.bar(x + 0.2, results_df['Test RMSE'], width=0.2, label='RMSE', alpha=0.8)

ax2.set_xlabel('Fold')
ax2.set_ylabel('Error')
ax2.set_title('Test Metrics by Fold')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Plot 3: Prediction vs Actual for a sample fold
sample_fold_idx = 0
sample_result = fold_results[sample_fold_idx]

# Take first 100 predictions for visualization
n_samples = 100
predictions = sample_result['predictions'][:n_samples]
targets = sample_result['targets'][:n_samples]

fig, axes = plt.subplots(2, 1, figsize=(16, 8))

# Plot first horizon step
ax1 = axes[0]
ax1.plot(targets[:, 0], label='Actual', linewidth=2, alpha=0.8)
ax1.plot(predictions[:, 0], label='Predicted', linewidth=2, alpha=0.8)
ax1.set_ylabel('Price Return')
ax1.set_title(f'Fold {sample_fold_idx + 1}: 1-Step Ahead Forecast (t+1)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot last horizon step
ax2 = axes[1]
ax2.plot(targets[:, -1], label='Actual', linewidth=2, alpha=0.8)
ax2.plot(predictions[:, -1], label='Predicted', linewidth=2, alpha=0.8)
ax2.set_xlabel('Sample Index')
ax2.set_ylabel('Price Return')
ax2.set_title(f'Fold {sample_fold_idx + 1}: {config.pred_len}-Step Ahead Forecast (t+{config.pred_len})')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 11. Regime-Specific Performance Analysis

Analyze model performance broken down by market regime.

In [ ]:
# Analyze performance by regime for first fold
fold_loader = fold_loaders[0]
model = create_dlinear_model(
    input_size=input_size,
    d_model=64,
    kernel_size=25,
    pred_len=config.pred_len,
    num_paths=1
).to(device)

# Quick train
history = train_model(model, fold_loader.train_dl, fold_loader.val_dl, epochs=10, lr=1e-3)

# Evaluate by regime
model.eval()
regime_errors = {}

with torch.no_grad():
    for batch in fold_loader.test_dl:
        features = batch['features'].to(device)
        targets = batch['targets'].to(device)
        regimes = batch.get('regime', None)
        
        predictions = model(features)
        
        if predictions.dim() == 3 and targets.dim() == 2:
            predictions = predictions.squeeze(-1)
        
        errors = torch.abs(predictions - targets).cpu().numpy()
        
        if regimes is not None:
            regimes = regimes.cpu().numpy()
            for regime_id in np.unique(regimes):
                mask = regimes == regime_id
                regime_errors.setdefault(regime_id, []).extend(errors[mask].flatten())

# Plot regime-specific errors
if regime_errors:
    fig, ax = plt.subplots(figsize=(12, 6))
    
    regime_labels = [f"Regime {k}" for k in sorted(regime_errors.keys())]
    regime_mae = [np.mean(regime_errors[k]) for k in sorted(regime_errors.keys())]
    
    bars = ax.bar(regime_labels, regime_mae, color=['green', 'yellow', 'red'][:len(regime_labels)], alpha=0.7)
    ax.set_ylabel('Mean Absolute Error')
    ax.set_title('Model Performance by Market Regime (Fold 1 Test Set)')
    ax.grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for bar, mae in zip(bars, regime_mae):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{mae:.4f}',
                ha='center', va='bottom', fontsize=11)
    
    plt.tight_layout()
    plt.show()
    
    print("\nRegime-Specific Performance:")
    for regime_id in sorted(regime_errors.keys()):
        print(f"  Regime {regime_id}: MAE = {np.mean(regime_errors[regime_id]):.6f}")
else:
    print("No regime labels available for analysis")

## 12. Save Results and Model

In [ ]:
# Save results
output_dir = Path('outputs/walkforward_dlinear')
output_dir.mkdir(parents=True, exist_ok=True)

# Save results dataframe
results_df.to_csv(output_dir / 'fold_results.csv', index=False)
print(f"Saved results to {output_dir / 'fold_results.csv'}")

# Optionally save model from last fold
model_path = output_dir / f'dlinear_fold_{num_folds}.pt'
torch.save(model.state_dict(), model_path)
print(f"Saved model to {model_path}")

print("\nAll outputs saved successfully!")

## Summary

This notebook demonstrated:

1. **Walk-Forward Validation**: Realistic temporal splitting to avoid lookahead bias
2. **Regime Detection**: Automatic clustering of market states for adaptive modeling
3. **DLinear Architecture**: Efficient decomposition-based time series forecasting
4. **Per-Fold Training**: Separate models per fold simulating production retraining
5. **Comprehensive Evaluation**: Metrics across folds and regimes

### Next Steps:
- Experiment with different regime clustering strategies (GMM, HDBSCAN)
- Add probabilistic forecasting (multi-path predictions with CRPS loss)
- Implement regime drift monitoring for live deployment
- Try different DLinear kernel sizes for seasonal decomposition
- Extend to multi-asset modeling with cross-asset features